In [4]:
import spacy
import os
import xml.etree.ElementTree as ET
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from spacy import displacy

In [2]:
from spacy.tokens import Doc
nlp = spacy.load(os.path.join("..", "sv_model_upos", "sv_model0", "sv_model_upos0-0.0.0"))

In [14]:
doc = nlp("Den snabba bruna räven hoppar över den lata hunden.")
for token in doc:
    print(token.text, token.tag_, token.dep_, token.is_stop,[child for child in token.children])

Den DET det True []
snabba ADJ amod False []
bruna ADJ amod False []
räven NOUN nsubj False [Den, snabba, bruna]
hoppar VERB ROOT False [räven, hunden, .]
över ADP case True []
den DET det True []
lata ADJ amod False []
hunden NOUN obl False [över, den, lata]
. PUNCT punct False []


In [13]:
displacy.render(doc, style="dep", jupyter=True)

In [21]:
for token in doc: 
    print(token, [
        ('((%s))' if t is token else '%s') % t.text
        for t in token.subtree
    ])

Den ['((Den))']
snabba ['((snabba))']
bruna ['((bruna))']
räven ['Den', 'snabba', 'bruna', '((räven))']
hoppar ['Den', 'snabba', 'bruna', 'räven', '((hoppar))', 'över', 'den', 'lata', 'hunden', '.']
över ['((över))']
den ['((den))']
lata ['((lata))']
hunden ['över', 'den', 'lata', '((hunden))']
. ['((.))']


In [38]:
for sent in doc.sents:
    print(sent.root, list(sent.root.lefts), list(list(sent.root.lefts)[0].children))

hoppar [räven] [Den, snabba, bruna]


In [54]:
def processDoc(doc):
    sentence = list(doc.sents)[0]
    root = sentence.root
    print("Sentence: ", sentence)
    print("Root: ", root)
    recurseTokens(root)
      
def recurseTokens(token): 
    print("Processing token: ", token)
    if(list(token.children) == []): 
        print("Leaf node")
        return token
    print("Left tokens: ", [recurseTokens(t) for t in token.lefts], "of token", token)
    print("Right tokens: ", [recurseTokens(t) for t in token.rights], "of token", token)
    
    

In [55]:
processDoc(doc)

Sentence:  Den snabba bruna räven hoppar över den lata hunden.
Root:  hoppar
Processing token:  hoppar
Processing token:  räven
Processing token:  Den
Leaf node
Processing token:  snabba
Leaf node
Processing token:  bruna
Leaf node
Left tokens:  [Den, snabba, bruna] of token räven
Right tokens:  [] of token räven
Left tokens:  [None] of token hoppar
Processing token:  hunden
Processing token:  över
Leaf node
Processing token:  den
Leaf node
Processing token:  lata
Leaf node
Left tokens:  [över, den, lata] of token hunden
Right tokens:  [] of token hunden
Processing token:  .
Leaf node
Right tokens:  [None, .] of token hoppar
